<a href="https://colab.research.google.com/github/DenisR144/healthdata22/blob/main/Dataton_2022_(DR_and_T_SH).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Основные действия по основной информации**


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Подключение основных библиотек**

In [197]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import sqlite3 as db
#import re
!pip install pymorphy2
import pymorphy2
import pymystem3
from collections import Counter
import csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Создание основной БД (без текстовых полей)**

In [258]:
#UID;Date;Name;DOB;MKB_1;MKB_2;MKB_3;Died;Violations;ViolationsCausedDeath;Hospital;Ambulance;Home;Meddoc
#df = pd.read_csv("drive/MyDrive/DataMed.csv")
#print(df)


def open_csv_file(csv_file_path):
    """
    Open and read data from a csv file without headers (skipping the first row)
    :param csv_file_path: path of the csv file to process
    :return: a list with the csv content
    """
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)

        data = list()
        for row in reader:
            data.append(row)

        return data

conn = db.connect(':memory:')
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS table1")

create_table = """CREATE TABLE table1(
                uid INTEGER PRIMARY KEY AUTOINCREMENT,
                date DATE NOT NULL,
                name VARCHAR(10) NOT NULL,
                dob DATE NOT NULL,
                mkb1 VARCHAR(1) NOT NULL,
                mkb2 INTEGER NOT NULL,
                mkb3 INTEGER NOT NULL,
                died INTEGER NOT NULL,
                violations INTEGER NOT NULL,
                vcd INTEGER NOT NULL,
                hospital INTEGER NOT NULL,
                ambulance INTEGER NOT NULL,
                home INTEGER NOT NULL,
                meddoc VARCHAR(50) NOT NULL,
                perc_of_noun FLOAT64 NULL,
                perc_of_adj FLOAT64 NULL,
                perc_of_verb FLOAT64 NULL,
                alcohol_num INT NULL,
                tabacco_num INT NULL
                );"""
 
cur.execute(create_table)

values_to_insert = open_csv_file('drive/MyDrive/DataMed.csv')
for vv in values_to_insert:
  insert_records = """INSERT INTO table1(uid,date,name,dob,mkb1,mkb2,mkb3,died,violations,vcd,hospital,ambulance,home,meddoc) 
  VALUES(
    """+vv[0].split(';')[0]+""",
    """+vv[0].split(';')[1]+""",
    '"""+vv[0].split(';')[2]+"""',
    """+vv[0].split(';')[3]+""",
    '"""+vv[0].split(';')[4]+"""',
    """+vv[0].split(';')[5]+""",
    """+vv[0].split(';')[6]+""",
    """+vv[0].split(';')[7]+""",
    """+vv[0].split(';')[8]+""",
    """+vv[0].split(';')[9]+""",
    """+vv[0].split(';')[10]+""",
    """+vv[0].split(';')[11]+""",
    """+vv[0].split(';')[12]+""",
    '"""+vv[0].split(';')[13]+"""'
    )
  """
  cur.execute(insert_records)
  #print(insert_records)




**Подгрузка текстов**

In [101]:
def parse_meddoc(filename : str): #-> perc_of_noun,perc_of_adj,perc_of_verb,alcohol_num,tabacco_num
  myfile = open(filename,'r') 
  words = []

  try:
    filelines = myfile.readlines()
    for line in filelines:
      words += line.split(' ')

  finally:
    f.close()

  words2 = words
  words3 = []

  # 1 - удаление специальных символов

  reg = re.compile('[^a-zA-Z]')
  n = 0
  for val in words2:
    val = val.replace(" ","")
    val = val.replace(".","")
    val = val.replace(",","")
    val = val.replace(":","")
    val = val.replace(";","")
    val = val.replace("1","")
    val = val.replace("2","")
    val = val.replace("3","")
    val = val.replace("4","")
    val = val.replace("5","")
    val = val.replace("6","")
    val = val.replace("7","")
    val = val.replace("8","")
    val = val.replace("9","")
    val = val.replace("0","")
    val = val.replace("(","")
    val = val.replace(")","")
    if len(val)>1: #отсекаем пустые сроки, и односимвольные
      words3.append(val)

  print("Strings are affected")



  # Лемматизация
  # Рабочий массив данных - список words3

  words5 = words3
  words6 = []
  words_pos = []

  m1 = pymorphy2.MorphAnalyzer()

  n=0
  for val in words5:
    p = m1.parse(val)[0]
    words6.append(p.normal_form)
    words_pos.append(p.tag.POS)
    n+=1

  print(str(n) + " words affected!")


  # считаем количество слов по частям речи

  c = Counter(words_pos)
  
  perc_of_noun = c['NOUN'] / len(words_pos) # доля существительных
  perc_of_adj = c['ADJF'] / len(words_pos) # доля прилагательных
  perc_of_verb = c['VERB'] / len(words_pos) # доля глаголов

  # считаем количество слов по гипотезам (табак, алкоголь)

  c2 = Counter(words6)
  alcohol_num = c2['спирт'] + c2['выпивка'] + c2['алкоголь'] + c2['выпивать'] + c2['выпить'] + c2['пить'] + c2['водка'] + c2['пиво'] + c2['употреблять']
  tabacco_num = c2['табак'] + c2['сигарета'] + c2['курить'] + c2['выкуривать']

  return(perc_of_noun,perc_of_adj,perc_of_verb,alcohol_num,tabacco_num)



**Обработка текстов**

1) Стемминг (отрезаем в словах окончания и суффиксы), есть вроде готовые библиотеки под русский язык //минусы - теряем часть информации, а часть - сливается

*или* 

2) Лемматизация (морфологический анализ) с целью логического объединения разных форм (например "шёл", "идти" - перевод в слов в начальные формы) - библиотеки pymorphy2, pymystem3

3) частотный анализ, т.е. закидываем все слова с частотным анализом в матрицу, и дальше обрабатываем как вектора
Это надо делать после выше приведенных 1 или 2

4) составление матрицы по частоте использования частей речи (глагол, существительное, прилагательное), перевод их в процентное соотношение от объема текста

1. избавимся от всех лишних символов, кроме русских букв, дефисов и пробелов.


2. цепочка дат событий: приведение дат к единому формату, выстраивание цепочек

3. создание словаря - значимых терминов (от Дениса Рощина, максимальная вероятность получить нормальный словарь)
далее слова из этого словаря найти в текстах и по каждой истории болезни вывести частотную характеристику по каждому слову словаря
В словаре могут быть синонимы.
Статистику по синонимам объединить.

Со словарем значимых слов мне кажется будет решение изящнее...

In [270]:

res = cur.execute("SELECT uid,meddoc FROM table1")
row = res.fetchall()
row

for a in row:
  aa = parse_meddoc(a[1]) #perc_of_noun,perc_of_adj,perc_of_verb,alcohol_num,tabacco_num
  res2 = cur.execute("UPDATE table1 SET perc_of_noun = '"+str(aa[0])+"', perc_of_adj = '"+str(aa[1])+"',perc_of_verb = '"+str(aa[2])+"',alcohol_num = '"+str(aa[3])+"',tabacco_num = '"+str(aa[4])+"' WHERE UID="+str(a[0]))

res = cur.execute("SELECT * FROM table1")
row = res.fetchall()
row

Strings are affected
31723 words affected!
Strings are affected
50871 words affected!
Strings are affected
26670 words affected!


[(1,
  2004,
  'GGT',
  1993,
  'C',
  61,
  2,
  1,
  0,
  0,
  1,
  0,
  1,
  '1.txt',
  0.5193392806481102,
  0.21375658039907952,
  0.03574693440090786,
  0,
  0),
 (2,
  1999,
  'ATM',
  1991,
  'О',
  26,
  8,
  1,
  0,
  0,
  1,
  1,
  0,
  '2.txt',
  0.49448605295748066,
  0.19567140413988324,
  0.04680466277446876,
  5,
  0),
 (3,
  2008,
  'CAM',
  1914,
  'I',
  63,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  '3.txt',
  0.4752530933633296,
  0.2116610423697038,
  0.03880764904386952,
  0,
  0)]

In [271]:
res = cur.execute("PRAGMA table_info(table1);")
row = res.fetchall()


row

[(0, 'uid', 'INTEGER', 0, None, 1),
 (1, 'date', 'DATE', 1, None, 0),
 (2, 'name', 'VARCHAR(10)', 1, None, 0),
 (3, 'dob', 'DATE', 1, None, 0),
 (4, 'mkb1', 'VARCHAR(1)', 1, None, 0),
 (5, 'mkb2', 'INTEGER', 1, None, 0),
 (6, 'mkb3', 'INTEGER', 1, None, 0),
 (7, 'died', 'INTEGER', 1, None, 0),
 (8, 'violations', 'INTEGER', 1, None, 0),
 (9, 'vcd', 'INTEGER', 1, None, 0),
 (10, 'hospital', 'INTEGER', 1, None, 0),
 (11, 'ambulance', 'INTEGER', 1, None, 0),
 (12, 'home', 'INTEGER', 1, None, 0),
 (13, 'meddoc', 'VARCHAR(50)', 1, None, 0),
 (14, 'perc_of_noun', 'FLOAT64', 0, None, 0),
 (15, 'perc_of_adj', 'FLOAT64', 0, None, 0),
 (16, 'perc_of_verb', 'FLOAT64', 0, None, 0),
 (17, 'alcohol_num', 'INT', 0, None, 0),
 (18, 'tabacco_num', 'INT', 0, None, 0)]

**Основной анализ обработанного текста и основных полей**

**Всяческая визуализация**